In [1]:
import sys
import json
from datetime import datetime
import torch
import numpy as np
import pandas as pd
from pathlib import Path
from typing import Dict, List, Any, Generator
from tqdm import tqdm
from collections import Counter
import random
import torch.nn as nn
import torchmetrics as tm
main_path = Path('..').resolve()
sys.path.append(str(main_path))

In [2]:
ld_data = pd.read_csv("../data/LD2011_2014.txt", sep="\;", engine="python", encoding="cp949")

In [3]:
ld_data.head(1000)

,"""""","""MT_001""","""MT_002""","""MT_003""","""MT_004""","""MT_005""","""MT_006""","""MT_007""","""MT_008""","""MT_009""",...,"""MT_361""","""MT_362""","""MT_363""","""MT_364""","""MT_365""","""MT_366""","""MT_367""","""MT_368""","""MT_369""","""MT_370"""
0,"""2011-01-01 00:15:00""",0,0,0,0,0,0,0,0,0,...,0,0.0,0,0,0,0,0,0,0,0
1,"""2011-01-01 00:30:00""",0,0,0,0,0,0,0,0,0,...,0,0.0,0,0,0,0,0,0,0,0
2,"""2011-01-01 00:45:00""",0,0,0,0,0,0,0,0,0,...,0,0.0,0,0,0,0,0,0,0,0
3,"""2011-01-01 01:00:00""",0,0,0,0,0,0,0,0,0,...,0,0.0,0,0,0,0,0,0,0,0
4,"""2011-01-01 01:15:00""",0,0,0,0,0,0,0,0,0,...,0,0.0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,"""2011-01-11 09:00:00""",0,0,0,0,0,0,0,0,0,...,0,0.0,0,0,0,0,0,0,0,0
996,"""2011-01-11 09:15:00""",0,0,0,0,0,0,0,0,0,...,0,0.0,0,0,0,0,0,0,0,0
997,"""2011-01-11 09:30:00""",0,0,0,0,0,0,0,0,0,...,0,0.0,0,0,0,0,0,0,0,0
998,"""2011-01-11 09:45:00""",0,0,0,0,0,0,0,0,0,...,0,0.0,0,0,0,0,0,0,0,0


In [4]:
ld_data.rename(columns = {'""':'date'}, inplace=True)

In [5]:
ld_data['date'] = ld_data['date'].str.strip('"')

In [6]:
ld_data['date'] = pd.to_datetime(ld_data['date'])

In [7]:
ld_data2 = ld_data.set_index('date')

In [8]:
col_list = [colname.strip('"') for colname in ld_data2.columns]

In [9]:
ld_data2.columns = col_list

In [10]:
for col in ld_data2.columns:
    if ld_data2[col].dtype != 'float64':
        ld_data2[col] = ld_data2[col].str.replace(",","").astype(float)

In [13]:
ld_data2 = ld_data2.loc[:].div(4)

In [28]:
ld_data2

,MT_001,MT_002,MT_003,MT_004,MT_005,MT_006,MT_007,MT_008,MT_009,MT_010,...,MT_361,MT_362,MT_363,MT_364,MT_365,MT_366,MT_367,MT_368,MT_369,MT_370
date,,,,,,,,,,,,,,,,,,,,,
2011-01-01 00:15:00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2011-01-01 00:30:00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2011-01-01 00:45:00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2011-01-01 01:00:00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2011-01-01 01:15:00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014-12-31 23:00:00,6.345178e+13,5.512091e+13,4.344049e+13,3.760163e+13,2.134146e+14,7.589286e+13,2.826456e+13,7.070707e+13,1.704545e+14,1.801075e+14,...,6.923626e+13,7050.0,4.040084e+13,3.409091e+13,7.496741e+12,1.462844e+14,1.742757e+14,4.424040e+13,1.627566e+14,1.905405e+14
2014-12-31 23:15:00,6.345178e+13,5.334282e+13,4.344049e+13,4.166667e+13,2.042683e+14,8.110119e+13,2.826456e+13,6.313131e+13,1.617133e+14,1.801075e+14,...,6.995004e+13,7075.0,3.924051e+13,3.352273e+13,7.496741e+12,2.486834e+14,1.679104e+14,4.215359e+13,1.673387e+14,1.675676e+13
2014-12-31 23:30:00,6.345178e+13,5.156472e+13,4.344049e+13,4.065041e+12,2.073171e+14,7.961310e+13,2.543810e+13,6.060606e+13,1.529720e+14,1.854839e+14,...,7.119914e+13,6950.0,3.892405e+13,3.295455e+13,6.844850e+13,2.340550e+13,1.676910e+14,3.839733e+13,1.675220e+14,1.716216e+14


In [30]:
for col in ld_data2.columns:
    data = ld_data2.loc[ld_data2[col] != 0][col].resample('H').sum()
    data.to_csv(f'./preprocessed/{col}.csv')

In [2]:
mt_001 = pd.read_csv("./preprocessed/MT_001.csv")

In [3]:
mt_001

,date,MT_001
0,2012-01-01 00:00:00,2.030457e+14
1,2012-01-01 01:00:00,2.982234e+14
2,2012-01-01 02:00:00,3.616751e+14
3,2012-01-01 03:00:00,2.157360e+14
4,2012-01-01 04:00:00,1.332487e+14
...,...,...
26300,2014-12-31 20:00:00,2.220812e+14
26301,2014-12-31 21:00:00,2.538071e+14
26302,2014-12-31 22:00:00,1.903553e+14
26303,2014-12-31 23:00:00,2.220812e+14


In [4]:
# Without meta learning
def sliding_window_idx(df_single, window_size, horizon):
    single_entity = []
    len_df = len(df_single)
    if  len_df >= window_size:
        for i in range(len_df-window_size-horizon+1):
                task_set = []
                x_idx = [idx for idx in range(i, i+window_size)]
                y_idx = [i+window_size+horizon-1]
                task_set.append(x_idx)
                task_set.append(y_idx)
                single_entity.append(task_set)

    return single_entity

In [5]:
single_entity  = sliding_window_idx(df_single = mt_001, window_size = 168, horizon = 3)

In [7]:
single_entity[-1]

[[26134,
  26135,
  26136,
  26137,
  26138,
  26139,
  26140,
  26141,
  26142,
  26143,
  26144,
  26145,
  26146,
  26147,
  26148,
  26149,
  26150,
  26151,
  26152,
  26153,
  26154,
  26155,
  26156,
  26157,
  26158,
  26159,
  26160,
  26161,
  26162,
  26163,
  26164,
  26165,
  26166,
  26167,
  26168,
  26169,
  26170,
  26171,
  26172,
  26173,
  26174,
  26175,
  26176,
  26177,
  26178,
  26179,
  26180,
  26181,
  26182,
  26183,
  26184,
  26185,
  26186,
  26187,
  26188,
  26189,
  26190,
  26191,
  26192,
  26193,
  26194,
  26195,
  26196,
  26197,
  26198,
  26199,
  26200,
  26201,
  26202,
  26203,
  26204,
  26205,
  26206,
  26207,
  26208,
  26209,
  26210,
  26211,
  26212,
  26213,
  26214,
  26215,
  26216,
  26217,
  26218,
  26219,
  26220,
  26221,
  26222,
  26223,
  26224,
  26225,
  26226,
  26227,
  26228,
  26229,
  26230,
  26231,
  26232,
  26233,
  26234,
  26235,
  26236,
  26237,
  26238,
  26239,
  26240,
  26241,
  26242,
  26243,
  26244,
 

In [26]:
len(single_entity[-1][0])

168

In [16]:
len(single_entity)

26138

In [28]:
# With meta learning
def meta_sliding_window_idx(df_single, window_size, horizon, n_support, n_query):
    if len(df_single) >= window_size:
        x_spt_task = []
        y_spt_task = []
        x_qry_task = []
        y_qry_task = []

        for i in range(len_df-window_size-horizon-n_support-n_query+1):
            x_spt = []
            y_spt = []
            x_qry = []
            y_qry = []

            for j in range(n_support+n_query):
                if j < n_support:
                    spt_idx = [idx for idx in range(i+j, i+j+window_size)]
                    x_spt.append(spt_idx)
                    y_spt.append(i+j+window_size+horizon-1)

                else:
                    qry_idx = [idx for idx in range(i+j, i+j+window_size)]
                    x_qry.append(qry_idx)
                    y_qry.append(i+j+window_size+horizon-1)

            x_spt_task.append(x_spt)
            y_spt_task.append(y_spt)
            x_qry_task.append(x_qry)
            y_qry_task.append(y_qry)
        return x_spt_task, y_spt_task, x_qry_task, y_qry_task


In [32]:
x_spt_task, y_spt_task, x_qry_task, y_qry_task = meta_sliding_window_idx(df_single = mt_)

NameError: name 'mt_' is not defined